In [1]:
from langfuse_config import langfuse, langfuse_handler

In [2]:
from graph import graph


  0%|          | 0/3037 [00:00<?, ?it/s]

In [3]:
import pprint
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage

inputs = {
    "messages": [
        HumanMessage('what was the percentage change in the net cash from operating activities from 2008 to 2009'),
    ]
}

for error in graph.stream(inputs, config={"callbacks": [langfuse_handler]}):
    for key, value in error.items():
        print(f"Output from node '{key}':")
        print("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    print()
    print("---")
    print()

Output from node 'extract_question':
---
{ 'question': 'what was the percentage change in the net cash from operating '
              'activities from 2008 to 2009',
  'steps': ['extract_question']}

---

Output from node 'cheating_retriever':
---
{ 'documents': [ Document(id='Single_JKHY/2009/page_28.pdf-3', metadata={'qa': "{'question': 'what was the percentage change in the net cash from operating activities from 2008 to 2009', 'answer': '14.1%', 'explanation': '', 'ann_table_rows': [6], 'ann_text_rows': [], 'steps': [{'op': 'minus2-1', 'arg1': '206588', 'arg2': '181001', 'res': '25587'}, {'op': 'divide2-2', 'arg1': '#0', 'arg2': '181001', 'res': '14.1%'}], 'program': 'subtract(206588, 181001), divide(#0, 181001)', 'gold_inds': {'table_6': '2008 the net cash from operating activities of year ended june 30 2009 2008 is $ 206588 ; the net cash from operating activities of year ended june 30 2009 2008 is $ 181001 ; the net cash from operating activities of year ended june 30 2009 is $ 

In [4]:
dataset = langfuse.get_dataset("convfinqa-train")
from datetime import datetime

def calculate_error(input, actual, expected):
    try:
        expected_parsed = float(expected.replace('%', 'e-2'))
        actual_parsed = float(actual.replace('%', 'e-2'))
        error = actual_parsed - expected_parsed
    except:
        # For non numeric, short texts do exact comparison
        if len(expected) < 5:
            error = actual == expected
        else:
            # For non-numeric longer texts, use LLM produced similarity
            error = -1 # TODO: not implemented yet
    
    return error

for item in dataset.items:
    # Make sure your application function is decorated with @observe decorator to automatically link the trace
    with item.observe(
        run_name=f"{datetime.now().strftime('%Y%m%d%H%M%S')}",
        run_description="My first run",
        run_metadata={"model": "llama3"},
    ) as trace_id:
        # run your @observe() decorated application on the dataset item input
        # output = = .run(item.input)
        output = graph.invoke(inputs, config={"callbacks": [langfuse_handler]})
        answer = output['answer']
        comment = output['generation']

        # optionally, evaluate the output to compare different runs more easily
        langfuse.score(
            trace_id=trace_id,
            name="<example_eval>",
            # any float value
            value=calculate_error(item.input, answer, item.expected_output),
            comment=comment,  # optional, useful to add reasoning
        )
 
# Flush the langfuse client to ensure all data is sent to the server at the end of the experiment run
langfuse.flush()

KeyboardInterrupt: 